### INIT

In [ ]:
import sys
from pathlib import Path
from multiversx_sdk import ApiNetworkProvider, ProxyNetworkProvider
from multiversx_sdk.abi import Abi
import os
import importlib
from argparse import Namespace

os.environ["MX_DEX_ENV"] = "chainsim"

sys.path.append(str(Path.cwd().parent.parent.absolute()))
import config
importlib.reload(config)
from context import Context
from utils.utils_chain import WrapperAddress
from utils.utils_generic import get_logger
from tools.chain_simulator_connector import ChainSimulator, start_handler

SIMULATOR_URL = "http://localhost:8085"
SIMULATOR_API = "http://localhost:3001"

GENERATE_BLOCKS_URL = f"{SIMULATOR_URL}/simulator/generate-blocks"
GENERATE_BLOCKS_UNTIL_EPOCH_REACHED_URL = f"{SIMULATOR_URL}/simulator/generate-blocks-until-epoch-reached"
PROJECT_ROOT = Path.cwd().parent.parent
proxy = ProxyNetworkProvider(SIMULATOR_URL)
api = ApiNetworkProvider(SIMULATOR_API)
DOCKER_URL = PROJECT_ROOT / "docker"
logger = get_logger("fees-collector-upgraded")

if config.CURRENT_ENV.value == "chainsim":
    chain_sim = ChainSimulator()
    if not chain_sim.is_running():
        docker_path = config.HOME / "Projects/testing/full-stack-docker-compose/chain-simulator"
        state_path = config.DEFAULT_WORKSPACE / "states"
        args = Namespace(docker_path=str(docker_path), state_path=str(state_path))
        chain_sim, found_accounts = start_handler(args)
        print(f'Loaded {len(found_accounts)} accounts')

context = Context()

wasm_path = "https://github.com/multiversx/mx-exchange-sc/releases/download/v3.4.1-rc4/fees-collector.wasm"
abi = Abi.load(config.HOME / "Downloads/fees-collector.abi(4).json")
contract_code_hash = "4a7f6baf4aeebd1c9892b6bafd74ee3548b04be17d23fc0d554318015568d0a9"

### CHAIN SIM CONFIG - FEES COLLECTOR

In [ ]:
from tools.chain_simulator_connector import ChainSimulator, start_handler
from argparse import Namespace

docker_path = config.HOME / "Projects/testing/full-stack-docker-compose/chain-simulator"
state_path = config.DEFAULT_WORKSPACE / "states"
args = Namespace(docker_path=str(docker_path), state_path=str(state_path))
chain_sim = ChainSimulator(docker_path)

In [ ]:
USERS = [
        "erd146exyad7pn95pru78egj07nnyfgnyeaytxte33nxxd24g55uccgs77rr7d",
        "erd1emxytu3umnzm4k2cn2xmtppy8j3dm3lnsjhfzkul8gd5a4xxuk3qsl4xjw", 
        "erd1rv5twgkz5uatdvgk5ymzmgzmz38dxqh8agvlkt97mfun8hn4x4xqyptslm",
        "erd1njvcr0r89km6pexrxh0d6h36pkeuwr5j042e7l46l3mdlxvwhejsfz9w4n",
        "erd1ss6u80ruas2phpmr82r42xnkd6rxy40g9jl69frppl4qez9w2jpsqj8x97" # DEX OWNER
]

from contracts.simple_lock_energy_contract import SimpleLockEnergyContract
energy_contract: SimpleLockEnergyContract = context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0]
BASE_TOKEN = energy_contract.base_token

In [ ]:
import json
import subprocess
from time import sleep
from typing import Any

from utils.utils_chain import Account


def load_accounts_state(project_root: Path, addresses: list[str]) -> list[dict[str, Any]]:
    states = []
    
    for address in addresses:
        print(f"Loading state for {address}")
        user_path = f"0_{address}_0_chain_config_state.json"
        system_account_path = f"0_system_account_state_{address}.json"
        
        user_file = project_root / "states" / user_path
        system_file = project_root / "states" / system_account_path
        
        if user_file.exists():
            with open(user_file, "r") as file:
                user_state = json.load(file)
                if user_state:
                    print(f"Found {user_file.name}")
                    states.append(user_state)
                
        if system_file.exists():
            with open(system_file, "r") as file:
                system_state = json.load(file)
                if system_state:
                    print(f"Found {system_file.name}")
                    states.append(system_state)
            
    return states
    
def apply_states(proxy: ProxyNetworkProvider, states: list[dict[str, Any]]):
    for state in states:
        proxy.do_post_generic(f"{SIMULATOR_URL}/simulator/set-state", state)

# @pytest.fixture
def load_and_apply_state(proxy: ProxyNetworkProvider, project_root: Path, owner: str, users: list[str]):
    # Load and set state for all keys
    with open(project_root / "states" / "0_all_all_keys.json", "r") as file:
        retrieved_state = json.load(file)
        apply_states(proxy, [retrieved_state])

    # Load owner and users state
    accounts = [owner]
    accounts.extend(users)
    states = load_accounts_state(project_root, accounts)
    apply_states(proxy, states)
        

def advance_blocks(number_of_blocks: int):
    proxy.do_post_generic(f"{GENERATE_BLOCKS_URL}/{number_of_blocks}", {})

def advance_epoch(number_of_epochs: int):
    proxy.do_post_generic(f"{GENERATE_BLOCKS_URL}/{number_of_epochs * 20}", {})

def advance_to_epoch(epoch: int):
    proxy.do_post_generic(f"{GENERATE_BLOCKS_UNTIL_EPOCH_REACHED_URL}/{epoch}", {})

def users_init(found_users: list[str]) -> list[Account]:
    print(context.deployer_account.address.bech32())
    context.deployer_account.sync_nonce(context.network_provider.proxy)

    users = []
    for user in found_users:
        user_account = Account(pem_file=config.DEFAULT_ACCOUNTS)
        user_account.address = WrapperAddress(user)
        user_account.sync_nonce(context.network_provider.proxy)
        users.append(user_account)

    return users

In [ ]:
from utils.contract_data_fetchers import FeeCollectorContractDataFetcher
from contracts.router_contract import RouterContract
from contracts.simple_lock_energy_contract import SimpleLockEnergyContract
from contracts.fees_collector_contract import FeesCollectorContract
from contracts.pair_contract import PairContract
from utils.contract_retrievers import retrieve_pair_by_address

fees_collector_contract: FeesCollectorContract
fees_collector_contract = context.get_contracts(config.FEES_COLLECTORS)[0]

energy_contract: SimpleLockEnergyContract
router_contract: RouterContract
pair_contract: PairContract

pair_contract: PairContract = context.get_contracts(config.PAIRS_V2)[1]    # operating pair
mex_contract = context.get_contracts(config.PAIRS_V2)[0] # egldmex contract
energy_contract = context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0]
router_contract = context.deploy_structure.get_deployed_contract_by_index(config.ROUTER_V2, 0)

print(fees_collector_contract.address)
print(router_contract.address)
print(pair_contract.address)
print(mex_contract.address)


FUND USER

In [ ]:
from multiversx_sdk import TransferTransactionsFactory, TransactionsFactoryConfig
from utils.utils_generic import split_to_chunks

def compose_state_for_user(b32_user: str, amount: int) -> dict[str, Any]:
    return {
            "address": b32_user,
            "nonce": 0,
            "balance": str(amount),
            "username": "",
            "code": "",
            "developerReward": "0",
            "ownerAddress": "",
            "pairs": {}
        }

def fund_chain_sim_users_w_egld(users: list[str], amount: int):
    chain_sim.apply_states([[compose_state_for_user(user, amount) for user in users]])
    print(f'Funded {len(users)} users with {amount} EGLD')
    
def fund_chain_sim_users_w_esdt_from_mainnet(users: list[str], esdt: str, amount: int):
    from utils.utils_chain import dec_to_padded_hex
    from multiversx_sdk import ProxyNetworkProvider
    mainnet_proxy = ProxyNetworkProvider("https://gateway.multiversx.com")

    for user in users:
        current_entry = mainnet_proxy.get_account_storage_entry(WrapperAddress(user), f"ELRONDesdt{esdt}")
        if not current_entry:
            raise Exception("No entry found")

        header = current_entry.value.hex()[:2]
        new_entry = f"{header}{dec_to_padded_hex(len(dec_to_padded_hex(amount)) // 2 + 1)}{'00'}{dec_to_padded_hex(amount)}"

        chain_sim.apply_states([[{
                "address": user,
                "pairs": {
                    current_entry.key.encode().hex(): new_entry
                }
            }]])
    print(f'Funded {len(users)} users with {amount} {esdt}')

def fund_shadowfork_users_w_egld(users: list[str], amount: int):
    purse_account = Account(pem_file=config.DEFAULT_ACCOUNTS)
    purse_account.address = WrapperAddress(config.SHADOWFORK_FUNDING_ADDRESS)
    purse_account.sync_nonce(context.network_provider.proxy)

    chain_id = context.network_provider.proxy.get_network_config().chain_id
    tx_config = TransactionsFactoryConfig(chain_id=chain_id)

    transactions = []
    for user_address in users:
        address = WrapperAddress(user_address)
        factory = TransferTransactionsFactory(tx_config)
        transaction = factory.create_transaction_for_native_token_transfer(
            sender=purse_account.address,
            receiver=address,
            native_amount=10 ** 16,
        )
        transaction.nonce = purse_account.nonce
        transaction.signature = purse_account.sign_transaction(transaction)
        transactions.append(transaction)
        purse_account.nonce += 1
        
    transactions_chunks = split_to_chunks(transactions, 100)
    for transactions_chunk in transactions_chunks:
        num_sent, _ = context.network_provider.proxy.send_transactions(transactions_chunk)
        print(f"Sent {num_sent}/{len(transactions_chunk)} transactions")

    from time import sleep
    logger.info(f"Funded {len(users)} users with {amount} EGLD")
    logger.debug("Waiting for 40 seconds to ensure all funding transactions are processed...")
    sleep(40)

Upgrade

In [ ]:
from utils.utils_chain import base64_to_hex
from utils.utils_chain import WrapperAddress
from time import sleep


def fees_collector_upgrade(bytecode_path: Path, expected_code_hash: str = None):
    context.deployer_account.sync_nonce(context.network_provider.proxy)

    tx_hash = fees_collector_contract.contract_upgrade(context.deployer_account, context.network_provider.proxy, bytecode_path, 
                                         [], 
                                         no_init=True)

    if "localhost" in context.network_provider.proxy.url:
        chain_sim.advance_blocks(1)
    else:
        sleep(6)

    code_hash = context.network_provider.proxy.get_account(WrapperAddress(fees_collector_contract.address)).contract_code_hash.hex()
    if expected_code_hash:
        assert code_hash == expected_code_hash
    else:
        print(f"Code hash: {code_hash}")
    
    return tx_hash

Swap to MEX helpers

In [ ]:
from multiversx_sdk.abi import Abi
from utils.utils_chain import get_token_details_for_address
from utils.utils_chain import WrapperAddress as Address
from multiversx_sdk.abi import U64Value, StringValue

def get_available_amount_for_swap(token: str):
    _, amount, _ = get_token_details_for_address(token, fees_collector_contract.address, context.network_provider.proxy)
    current_week = fees_collector_contract.get_current_week(context.network_provider.proxy)
    locked_rewards = 0
    logger.debug(f"Current week: {current_week}")
    logger.debug(f"Token: {token} amount: {amount}")
    for week in range(current_week-4, current_week):
        logger.debug(f"Week: {week}")
        response = fees_collector_contract.get_total_rewards_for_week(context.network_provider.proxy, week, abi)
        for token_payment in response:
            if token_payment.token_identifier == token:
                locked_rewards += token_payment.amount
                logger.debug(f"{token} rewards for week: {locked_rewards}")
        claimed_rewards = fees_collector_contract.get_rewards_claimed(context.network_provider.proxy, week, token)
        locked_rewards -= claimed_rewards
        logger.debug(f"Claimed {token} rewards for week: {claimed_rewards}")

    accumulated_fees = fees_collector_contract.get_accumulated_fees(context.network_provider.proxy, token)
    logger.debug(f"Accumulated fees for {token}: {accumulated_fees}")

    return amount - locked_rewards - accumulated_fees

def get_available_amount_for_swap_via_view(token: str):
    week = fees_collector_contract.get_current_week(context.network_provider.proxy)
    data_fetcher = FeeCollectorContractDataFetcher(Address(fees_collector_contract.address), context.network_provider.proxy.url)
    result = data_fetcher.get_data("getTokenAvailableAmount", [U64Value(week), StringValue(token)])
    return result

def get_routes_for_swap(swapped_token: str):
    end_token = BASE_TOKEN

    # Find shortest path between swapped_token and end_token through pairs
    def find_shortest_path(start_token: str, end_token: str) -> list[tuple[str, str]]:
        # Build graph of token pairs
        pairs_graph = {}
        for pair in context.get_contracts(config.PAIRS_V2):
            token_a = pair.firstToken
            token_b = pair.secondToken
            
            if token_a not in pairs_graph:
                pairs_graph[token_a] = []
            if token_b not in pairs_graph:
                pairs_graph[token_b] = []
                
            pairs_graph[token_a].append((token_b, pair))
            pairs_graph[token_b].append((token_a, pair))

        # BFS to find shortest path
        queue = [(start_token, [])]
        visited = {start_token}
        
        while queue:
            current_token, path = queue.pop(0)
            
            if current_token == end_token:
                return path
                
            for next_token, pair in pairs_graph.get(current_token, []):
                if next_token not in visited:
                    visited.add(next_token)
                    queue.append((next_token, path + [(pair, current_token)]))
                    
        return [] # No path found

    # Get route of pairs to swap through
    path = find_shortest_path(swapped_token, end_token)
    if not path:
        raise Exception(f"No path found between {swapped_token} and {end_token}")
    
    router = []
    for pair, token in path:
        router.append((Address(pair.address), "swapTokensFixedInput", pair.firstToken if token == pair.secondToken else pair.secondToken, 1))
        
    return router
    
def build_args_for_swap(token: str, amount: int):
    routes: list[list[Any]] = []
    try:
        routes = get_routes_for_swap(token)
    except Exception as e:
        logger.error(f"Error getting routes for swap: {e}")
        return []
    
    sc_args = [
           token, 
           amount,
           routes
        ]

    return sc_args

def swap_all_tokens_to_base_token():
    context.deployer_account.sync_nonce(context.network_provider.proxy)
    tokens = context.deploy_structure.tokens    # TODO: get tokens from the network, but esdt route for fees collector fails (state too large)
    for token in tokens:
        if token == BASE_TOKEN:
            continue
        
        available_amount = get_available_amount_for_swap_via_view(token)
        if available_amount == 0:
            continue

        logger.debug(f"Swapping {available_amount} {token} to base token")
        args = build_args_for_swap(token, available_amount)
        if len(args) == 0:
            continue

        fees_collector_contract.swap_to_base_token(context.deployer_account, context.network_provider.proxy, abi, args)
    
    if config.CURRENT_ENV.value == "chainsim":
        chain_sim.advance_blocks(10)

Swaps for deposits

In [ ]:
from contracts.pair_contract import SwapFixedInputEvent
from multiversx_sdk import ApiNetworkProvider
from utils.utils_tx import endpoint_call
from utils.utils_chain import get_token_details_for_address
from utils.contract_data_fetchers import PairContractDataFetcher
from multiversx_sdk.abi import TokenIdentifierValue, BigUIntValue
from multiversx_sdk import Token

def swaps_for_deposits(swap_fraction: int):
    """swap_fraction is a percentage of the swappable balance to swap"""
    
    mainnet_api = ApiNetworkProvider("https://api.multiversx.com")
    
    hashes = []
    for token in context.deploy_structure.tokens:
        logger.debug("--------------------------------")
        logger.debug(f"Swapping for token {token}")
        # search for holder accounts and swap them
        holder_addresses_on_network = mainnet_api.do_get_generic(f"tokens/{token}/accounts")
        for entry in holder_addresses_on_network:
            swapping_address = WrapperAddress(entry.get("address"))

            if swapping_address.is_smart_contract():
                continue

            if config.CURRENT_ENV.value == "chainsim":
                address_roothash = context.network_provider.proxy.get_account(swapping_address).raw.get("account").get("rootHash")
                if address_roothash is None:
                    # we have to fill this account with egld and esdt
                    fund_chain_sim_users_w_egld([swapping_address.bech32()], 10 * 10**18)
                    mainnet_amount = mainnet_api.get_token_of_account(swapping_address, Token(token)).amount
                    fund_chain_sim_users_w_esdt_from_mainnet([swapping_address.bech32()], token, mainnet_amount)

            try:
                _, swappable_balance, _ = get_token_details_for_address(token, swapping_address.bech32(), context.network_provider.proxy)
                if swappable_balance == 0:
                    continue
            except Exception as e:
                logger.error(f"Error getting token details for {token} on {swapping_address.bech32()}: {e}")
                continue

            logger.debug(f"Found {swappable_balance} {token} on {swapping_address.bech32()}")

            swapping_account = Account(pem_file=config.DEFAULT_ACCOUNTS)
            swapping_account.address = swapping_address
            swapping_account.sync_nonce(context.network_provider.proxy)

            # fund the account if not enough egld for feess
            egld_balance = context.network_provider.proxy.get_account(swapping_account.address).balance
            if egld_balance < 1 * 10**16:
                purse_account = Account(pem_file=config.DEFAULT_ACCOUNTS)
                purse_account.address = WrapperAddress(config.SHADOWFORK_FUNDING_ADDRESS)
                purse_account.sync_nonce(context.network_provider.proxy)
                hash = endpoint_call(context.network_provider.proxy, 100000, purse_account, swapping_account.address, "", [], 1 * 10**18)
                context.network_provider.check_complex_tx_status(hash)
            
            # find the pair contract for the token
            swapping_pair_contract: PairContract = None
            for pair in context.get_contracts(config.PAIRS_V2):
                if pair.firstToken == token or pair.secondToken == token:
                    swapping_pair_contract = pair
                    break

            if swapping_pair_contract is None:
                logger.error(f"No pair contract found for token {token}")
                break
            
            # swap the tokens both ways
            swap_amount = swappable_balance * swap_fraction // 100

            def chain_sim_check_tx_status(hash: str):
                hashes.append(hash)

            if config.CURRENT_ENV.value == "chainsim":
                check_tx_status_fn = chain_sim_check_tx_status
            elif swapping_account.address.get_shard() != 1:
                check_tx_status_fn = context.network_provider.check_complex_tx_status
            else:
                check_tx_status_fn = context.network_provider.wait_for_tx_executed

            pair_data_fetcher = PairContractDataFetcher(WrapperAddress(swapping_pair_contract.address), context.network_provider.proxy.url)
            equivalent_amount = pair_data_fetcher.get_data("getAmountOut", [TokenIdentifierValue(token), BigUIntValue(swap_amount)])
            if equivalent_amount <= 0:
                logger.error(f"No equivalent swappable amount found for token {token} and swap amount {swap_amount}")
                continue
            
            other_token = swapping_pair_contract.secondToken if swapping_pair_contract.firstToken == token else swapping_pair_contract.firstToken
            event = SwapFixedInputEvent(token, swap_amount, other_token, 1)
            hash = swapping_pair_contract.swap_fixed_input(context.network_provider, swapping_account, event)
            check_tx_status_fn(hash)

            event = SwapFixedInputEvent(other_token, equivalent_amount, token, 1)
            hash = swapping_pair_contract.swap_fixed_input(context.network_provider, swapping_account, event)
            check_tx_status_fn(hash)

            break   # one swap per token is enough

    if config.CURRENT_ENV.value == "chainsim":
        blocks_to_advance = len(hashes) * 50_000_000 // 600_000_000 + 1 + 10
        print(f"Advancing {blocks_to_advance} blocks")
        chain_sim.advance_blocks(blocks_to_advance)
        for hash in hashes:
            context.network_provider.check_simple_tx_status(hash)

In [ ]:
def get_next_week_start_epoch(fees_collector_contract: FeesCollectorContract) -> int:
    first_week = fees_collector_contract.get_first_week_start_epoch(context.network_provider.proxy)
    current_week = fees_collector_contract.get_current_week(context.network_provider.proxy)
    next_week_at_epoch = first_week + current_week * 7

    print(f"Current epoch: {context.network_provider.proxy.get_network_status().current_epoch}")
    print(f"Current week: {current_week}")
    print(f"Next week at epoch: {next_week_at_epoch}")

    return next_week_at_epoch

def advance_to_next_week(fees_collector_contract: FeesCollectorContract):
    changing_epoch = get_next_week_start_epoch(fees_collector_contract)
    current_epoch = context.network_provider.proxy.get_network_status().current_epoch
    epochs_to_fast_forward = changing_epoch - current_epoch
    logger.info(f"Fast forwarding to {changing_epoch}")

    if config.CURRENT_ENV.value == "shadowfork4":
        from contracts.builtin_contracts import SFControlContract
        context.deployer_account.sync_nonce(context.network_provider.proxy)
        sf_control_contract = SFControlContract(config.SF_CONTROL_ADDRESS)
        sf_control_contract.epochs_fast_forward(context.deployer_account, context.network_provider.proxy, epochs_to_fast_forward, 9)

    elif config.CURRENT_ENV.value == "chainsim":
        chain_sim.advance_epochs_to_epoch(changing_epoch)
    else:
        raise Exception(f"Unknown environment: {config.CURRENT_ENV.value}")

    while context.network_provider.proxy.get_network_status().current_epoch < changing_epoch:
        sleep(6)
    logger.info(f"Fast forwarded to {changing_epoch}")


# CHAIN CONTROL

In [ ]:
chain_sim.stop()

In [ ]:
from tools.chain_simulator_connector import ChainSimulator, start_handler
from argparse import Namespace

docker_path = config.HOME / "Projects/testing/full-stack-docker-compose/chain-simulator"
state_path = config.DEFAULT_WORKSPACE / "states"
args = Namespace(docker_path=str(docker_path), state_path=str(state_path))
chain_sim = ChainSimulator(docker_path)

chain_sim, found_accounts = start_handler(args)
print(f'Loaded {len(found_accounts)} accounts')

In [ ]:
chain_sim = ChainSimulator("")

# CONTRACTS

In [ ]:
from utils.utils_chain import Account, WrapperAddress as Address

users = users_init()
user_account = users[0]
user_account.sync_nonce(context.network_provider.proxy)
print(user_account.address.to_bech32())

In [ ]:
import json
with open(PROJECT_ROOT / "states" / "0_erd1ss6u80ruas2phpmr82r42xnkd6rxy40g9jl69frppl4qez9w2jpsqj8x97_0_chain_config_state.json", "r") as file:
    json_account = json.load(file)
proxy.do_post_generic(f"{SIMULATOR_URL}/simulator/set-state", json_account)

In [ ]:
import json
with open(PROJECT_ROOT / "states" / "0_fees_collectors_0_chain_config_state.json", "r") as file:
    json_account = json.load(file)
proxy.do_post_generic(f"{SIMULATOR_URL}/simulator/set-state", json_account)

In [ ]:
chain_sim.advance_epochs(7)

In [ ]:
chain_sim.advance_blocks(5)

CLAIM REWARDS

In [ ]:
user_with_energy = users[1]
print(user_with_energy.address.to_bech32())
energy_contract.get_energy_for_user(context.network_provider.proxy, user_with_energy.address.to_bech32())

In [ ]:
user_without_energy = users[0]
print(user_without_energy.address.to_bech32())
energy_contract.get_energy_for_user(context.network_provider.proxy, user_without_energy.address.to_bech32())

In [ ]:
user_with_energy.sync_nonce(context.network_provider.proxy)

fees_collector_contract.claim_rewards(user_with_energy, proxy) #4 blocks
chain_sim.advance_blocks(5)

In [ ]:
user_without_energy.sync_nonce(context.network_provider.proxy)

fees_collector_contract.claim_rewards(user_without_energy, proxy) #4 blocks
chain_sim.advance_blocks(5)

In [ ]:
fees_collector_contract.claim_rewards(context.deployer_account, context.network_provider.proxy)

CLAIM BOOSTED

In [ ]:
fees_collector_contract.claim_boosted_rewards(user_account, proxy)

ADD ADMIN

In [ ]:
from utils.utils_chain import WrapperAddress

context.deployer_account.sync_nonce(proxy)
fees_collector_contract.add_admin(context.deployer_account, proxy, [context.deployer_account.address])

REMOVE ADMIN

In [ ]:
from utils.utils_chain import WrapperAddress

context.deployer_account.sync_nonce(proxy)
fees_collector_contract.remove_admin(context.deployer_account, proxy, [context.deployer_account.address])

REDISTRIBUTE REWARDS

In [ ]:

context.deployer_account.sync_nonce(proxy)
fees_collector_contract.redistribute_rewards(context.deployer_account, proxy)
advance_blocks(1)

SET ADMINS

In [ ]:
fees_collector_contract.add_admin(context.deployer_account, proxy, [context.deployer_account.address])

In [ ]:
week = fees_collector_contract.get_current_week(proxy)
user_energy = fees_collector_contract.get_user_energy_for_week(user_account.address.to_bech32(), proxy, week)
last_active_week = fees_collector_contract.get_last_active_week_for_user(user_account.address.to_bech32(), proxy)
print(week)
print(user_energy)
print(last_active_week)

GET TOTAL REWARDS FOR WEEK

In [ ]:
from utils.utils_chain import WrapperAddress, decode_merged_attributes
from utils import decoding_structures

data_fetcher = FeeCollectorContractDataFetcher(WrapperAddress(fees_collector_contract.address), context.network_provider.proxy.url)
total_rewards = fees_collector_contract.get_total_rewards_for_week(proxy, week)
print(total_rewards)


GET ACCUMULATED FEES

In [ ]:
from multiversx_sdk.abi import TokenIdentifierValue, U32Value
token_identifier = "MEX-455c57"
data_fetcher = FeeCollectorContractDataFetcher(WrapperAddress(fees_collector_contract.address), context.network_provider.proxy.url)
accumulated_fees = data_fetcher.get_data("getAccumulatedFees", [U32Value(week), TokenIdentifierValue(token_identifier)])
print(accumulated_fees)


GET USER ENERGY FOR WEEK

In [ ]:
data_fetcher = FeeCollectorContractDataFetcher(WrapperAddress(fees_collector_contract.address), context.network_provider.proxy.url)
user_energy_for_week = fees_collector_contract.get_user_energy_for_week(user_account.address.to_bech32(), proxy, week)
print(user_energy_for_week)


GET CURRENT CLAIM PROGRESS

In [ ]:
data_fetcher = FeeCollectorContractDataFetcher(WrapperAddress(fees_collector_contract.address), context.network_provider.proxy.url)
current_claim = fees_collector_contract.get_current_claim_progress_for_user(user_account.address.to_bech32(), proxy)
print(current_claim)

In [ ]:
mex_contract.whitelist_contract(context.deployer_account, context.network_provider.proxy, pair_contract.address)

In [ ]:
# set where to swap and what to do with the fees
pair_contract.add_trusted_swap_pair(context.deployer_account, context.network_provider.proxy,
                                    [
                                        mex_contract.address,
                                        mex_contract.firstToken,
                                        mex_contract.secondToken
                                    ])

In [ ]:
from contracts.pair_contract import AddLiquidityEvent

event = AddLiquidityEvent(pair_contract.firstToken, 127791780000000000000, 1, pair_contract.secondToken, 5000000000000000000, 1)
pair_contract.add_liquidity(context.network_provider, user_account, event)

In [ ]:
from utils.utils_chain import base64_to_hex


code_hash = context.network_provider.proxy.get_account(WrapperAddress(fees_collector_contract.address)).contract_code_hash.hex()
print(code_hash)

PAIR SWAP

In [ ]:
from utils.utils_chain import get_token_details_for_address
from contracts.pair_contract import PairContract, SwapFixedInputEvent

pair_contract: PairContract= context.get_contracts(config.PAIRS_V2)[1]
print(pair_contract.get_config_dict())

_, amount, _ = get_token_details_for_address(pair_contract.secondToken, user_account.address.to_bech32(), context.network_provider.proxy)
if amount == 0:
    raise Exception(f"No amount found on {user_account.address.to_bech32()}")
print(f'Amount found on {user_account.address.to_bech32()}: {amount}')

user_account.sync_nonce(proxy)
swap = SwapFixedInputEvent(pair_contract.secondToken, amount//2, pair_contract.firstToken, 1)

pair_contract.swap_fixed_input(context.network_provider, user_account, swap)
chain_sim.advance_blocks(5)

In [ ]:
from utils.utils_chain import get_token_details_for_address
from contracts.pair_contract import PairContract, SwapFixedInputEvent

pair_contract: PairContract= context.get_contracts(config.PAIRS_V2)[1]
print(pair_contract.get_config_dict())

_, amount, _ = get_token_details_for_address(pair_contract.firstToken, user_account.address.to_bech32(), context.network_provider.proxy)
if amount == 0:
    raise Exception(f"No amount found on {user_account.address.to_bech32()}")
print(f'Amount found on {user_account.address.to_bech32()}: {amount}')

user_account.sync_nonce(proxy)
swap = SwapFixedInputEvent(pair_contract.firstToken, amount, pair_contract.secondToken, 1)

pair_contract.swap_fixed_input(context.network_provider, user_account, swap)
chain_sim.advance_blocks(5)

SET ROUTER ADDRESS

In [ ]:
context.deployer_account.sync_nonce(proxy)
fees_collector_contract.set_router_address(context.deployer_account, proxy, router_contract.address)
chain_sim.advance_blocks(1)

SET BURN PERCENTAGES

In [ ]:
context.deployer_account.sync_nonce(proxy)
fees_collector_contract.set_base_token_burn_percent(context.deployer_account, proxy, 5000)
chain_sim.advance_blocks(1)

In [ ]:
pair_contract.add_fees_collector(context.deployer_account, proxy, [fees_collector_contract.address, 100000])
chain_sim.advance_blocks(1)

In [ ]:
print(pair_contract.get_config_dict())

SWAP TO BASE TOKEN

theoretical available amount

In [ ]:
token = pair_contract.secondToken
token = "A1X-0d446d"

In [ ]:
print(token)
amount = get_available_amount_for_swap(token)
print(f'calculated amount for swap: {amount}')

contract available amount

In [ ]:
result = get_available_amount_for_swap_via_view(token)
print(f'contract available amount: {result}')

In [ ]:
print(pair_contract.secondToken)
amount = get_available_amount_for_swap(pair_contract.secondToken)
print(f'available amount for swap: {amount}')
args = build_args_for_swap(pair_contract.secondToken, amount)
context.deployer_account.sync_nonce(context.network_provider.proxy)
fees_collector_contract.swap_to_base_token(context.deployer_account, context.network_provider.proxy, abi, args)
if config.CURRENT_ENV.value == "chainsim":
    chain_sim.advance_blocks(1)

In [ ]:
initial = amount // 2
print(f'will swap {initial} of {pair_contract.secondToken}; remaining {amount - initial}')
args = build_args_for_swap(pair_contract.secondToken,initial)
context.deployer_account.sync_nonce(context.network_provider.proxy)
fees_collector_contract.swap_to_base_token(context.deployer_account, context.network_provider.proxy, abi, args)
if config.CURRENT_ENV.value == "chainsim":
    chain_sim.advance_blocks(1)

In [ ]:
chain_sim.advance_blocks(1)

ADD REWARD TOKENS

In [ ]:
context.deployer_account.sync_nonce(proxy)
fees_collector_contract.add_reward_tokens(context.deployer_account, proxy, ["MEX-455c57", "XMEX-fda355"])

REMOVE REWARD TOKENS

In [ ]:
fees_collector_contract.remove_reward_tokens(context.deployer_account, proxy, ["USDC-c76f1f",
                                                                               "RIDE-7d18e9", 
                                                                               "CRU-a5f4aa", 
                                                                               "ZPAY-247875", 
                                                                               "ITHEUM-df6f26", 
                                                                               "BHAT-c1fde3", 
                                                                               "CRT-52decf", 
                                                                               "UTK-2f80e9",
                                                                               "QWT-46ac01",
                                                                               "ASH-a642d1",
                                                                               "WETH-b4ca29",
                                                                               "USDT-f8c08c",
                                                                               "HTM-f51d55",
                                                                               "WDAI-9eeb54",
                                                                               "TADA-5c032c",
                                                                               "XOXNO-c1293a",
                                                                               "A1X-0d446d",
                                                                               "USH-111e09",
                                                                               "FOXSY-5d5f3e"
                                                                               ])
advance_blocks(1)

In [ ]:
energy_factory: SimpleLockEnergyContract
energy_factory = context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0]

context.deployer_account.sync_nonce(proxy)
tx_hash = energy_factory.set_burn_role_locked_token(context.deployer_account,
                                                                proxy,
                                                                [fees_collector_contract.address])

advance_blocks(1)

In [ ]:
energy_factory.set_transfer_role_locked_token(context.deployer_account, context.network_provider.proxy, [mex_contract.address])
advance_blocks(1)

In [ ]:
from contracts.builtin_contracts import ESDTContract

esdt_contract = ESDTContract(config.TOKENS_CONTRACT_ADDRESS)
context.deployer_account.sync_nonce(context.network_provider.proxy)
tx_hash = esdt_contract.set_special_role_token(context.deployer_account, context.network_provider.proxy,
                                               [BASE_TOKEN, fees_collector_contract.address, "ESDTRoleLocalMint", "ESDTRoleLocalBurn"])


# SCENARIOS

In [ ]:
from tools.chain_simulator_connector import ChainSimulator, start_handler
from utils.utils_chain import get_token_details_for_address, WrapperAddress as Address
from contracts.pair_contract import PairContract, SwapFixedInputEvent
from argparse import Namespace
from time import sleep

docker_path = config.CHAIN_SIMULATOR_DOCKER_PATH
state_path = config.DEFAULT_WORKSPACE / "states"
args = Namespace(docker_path=str(docker_path), state_path=str(state_path))
chain_sim, found_accounts = start_handler(args)
print(f'Loaded {len(found_accounts)} accounts')
sleep(10)

In [ ]:
from tools.chain_simulator_connector import ChainSimulator, start_handler
from utils.utils_chain import get_token_details_for_address, WrapperAddress as Address
from contracts.pair_contract import PairContract, SwapFixedInputEvent
from contracts.builtin_contracts import ESDTContract

# INITIALIZE USERS

USERS = found_accounts
users = users_init()

purse_account = next(user for user in users if user.address.to_bech32() == "erd146exyad7pn95pru78egj07nnyfgnyeaytxte33nxxd24g55uccgs77rr7d")
user_without_energy = next(user for user in users if user.address.to_bech32() == "erd146exyad7pn95pru78egj07nnyfgnyeaytxte33nxxd24g55uccgs77rr7d")
user_with_energy = next(user for user in users if user.address.to_bech32() == "erd1emxytu3umnzm4k2cn2xmtppy8j3dm3lnsjhfzkul8gd5a4xxuk3qsl4xjw")
user_with_energy_2 = next(user for user in users if user.address.to_bech32() == "erd1adljw932qra4sf5mpxjyzelmf4lykwt5ppxlre59utjcpc22uhms2qxcqx")

pair_contract: PairContract= context.get_contracts(config.PAIRS_V2)[1]
print(pair_contract.get_config_dict())

_, amount, _ = get_token_details_for_address(pair_contract.secondToken, purse_account.address.to_bech32(), context.network_provider.proxy)
if amount == 0:
    raise Exception(f"No amount found on {purse_account.address.to_bech32()}")
print(f'Amount found on {purse_account.address.to_bech32()}: {amount}')

# GET KNOWN CONTRACTS BEFORE UPGRADE -- THESE WILL BE REMOVED BY THE UPGRADE
known_contracts = fees_collector_contract.get_known_contracts(proxy)

# UPGRADE CONTRACT

fees_collector_upgrade()

context.deployer_account.sync_nonce(proxy)
fees_collector_contract.set_router_address(context.deployer_account, proxy, router_contract.address)
chain_sim.advance_blocks(1)

fees_collector_contract.add_admin(context.deployer_account, context.network_provider.proxy, [context.deployer_account.address])
chain_sim.advance_blocks(1)

# BEGIN SCENARIO

def week_activity():
    # CLAIM REWARDS 1

    user_with_energy.sync_nonce(context.network_provider.proxy)
    fees_collector_contract.claim_rewards(user_with_energy, proxy) #4 blocks
    chain_sim.advance_blocks(5)

    user_without_energy.sync_nonce(context.network_provider.proxy)
    fees_collector_contract.claim_rewards(user_without_energy, proxy) #4 blocks
    chain_sim.advance_blocks(5)

    # SWAP

    purse_account.sync_nonce(proxy)
    swap = SwapFixedInputEvent(pair_contract.secondToken, amount//10, pair_contract.firstToken, 1)

    pair_contract.swap_fixed_input(context.network_provider, purse_account, swap)
    chain_sim.advance_blocks(5)

    # CONVERT TO MEX
    swappable_amount = get_available_amount_for_swap(pair_contract.secondToken)
    args = build_args_for_swap(pair_contract.secondToken, swappable_amount)
    context.deployer_account.sync_nonce(proxy)
    fees_collector_contract.swap_to_base_token(context.deployer_account, proxy, abi, args)
    chain_sim.advance_blocks(1)

    # CLAIM REWARDS 2

    user_with_energy_2.sync_nonce(context.network_provider.proxy)
    fees_collector_contract.claim_rewards(user_with_energy_2, proxy) #4 blocks
    chain_sim.advance_blocks(5)

# FIRST WEEKS ------------------------------------------------------------

weeks = 5
for week in range(weeks):

    print(f"Week {week}")

    # WEEK ACTIVITY
    week_activity()

    # PASS THE WEEK
    chain_sim.advance_epochs_to_epoch(get_next_week_start_epoch(fees_collector_contract))

# WEEK 5 SETUP ------------------------------------------------------------

user_with_energy.sync_nonce(context.network_provider.proxy)
fees_collector_contract.claim_rewards(user_with_energy, proxy) #4 blocks
chain_sim.advance_blocks(5)

# CLEANUP KNOWN TOKENS

tokens = fees_collector_contract.get_reward_tokens(proxy)
tokens.remove(energy_contract.base_token)
tokens.remove(energy_contract.locked_token)

fees_collector_contract.remove_reward_tokens(context.deployer_account, proxy, tokens)
chain_sim.advance_blocks(1)

# SET NEW BURN PERCENTAGE

esdt_contract = ESDTContract(config.TOKENS_CONTRACT_ADDRESS)
esdt_contract.set_special_role_token(context.deployer_account, context.network_provider.proxy,
                                               [energy_contract.base_token, fees_collector_contract.address, "ESDTRoleLocalBurn"])

chain_sim.advance_blocks(5)

fees_collector_contract.set_base_token_burn_percent(context.deployer_account, proxy, 5000)
chain_sim.advance_blocks(1)

# MODIFY FEES FOR KNOWN CONTRACTS

for address in known_contracts:
    mod_contract = PairContract.load_contract_by_address(address)
    if mod_contract is None:
        continue
    mod_contract.add_fees_collector(context.deployer_account, proxy, [fees_collector_contract.address, 100000])
    chain_sim.advance_blocks(1)

# WEEK ACTIVITY
week_activity()

chain_sim.advance_epochs_to_epoch(get_next_week_start_epoch(fees_collector_contract))

# WEEK ACTIVITY
week_activity()


SCENARIO INIT

In [ ]:
from utils.utils_scenarios import PhaseDictsCollector
from tools.chain_simulator_connector import ChainSimulator, start_handler
from utils.utils_chain import get_token_details_for_address, WrapperAddress as Address
from contracts.pair_contract import PairContract, SwapFixedInputEvent
from contracts.builtin_contracts import ESDTContract
from argparse import Namespace
from time import sleep

def test_scenario(bytecode_path: Path, collector: PhaseDictsCollector):
    users = users_init(found_accounts)

    purse_account = next(user for user in users if user.address.to_bech32() == "erd146exyad7pn95pru78egj07nnyfgnyeaytxte33nxxd24g55uccgs77rr7d")
    user_without_energy = next(user for user in users if user.address.to_bech32() == "erd146exyad7pn95pru78egj07nnyfgnyeaytxte33nxxd24g55uccgs77rr7d")
    user_with_energy = next(user for user in users if user.address.to_bech32() == "erd1emxytu3umnzm4k2cn2xmtppy8j3dm3lnsjhfzkul8gd5a4xxuk3qsl4xjw")
    user_with_energy_2 = next(user for user in users if user.address.to_bech32() == "erd1adljw932qra4sf5mpxjyzelmf4lykwt5ppxlre59utjcpc22uhms2qxcqx")

    pair_contract: PairContract= context.get_contracts(config.PAIRS_V2)[1]
    print(pair_contract.get_config_dict())

    _, amount, _ = get_token_details_for_address(pair_contract.secondToken, purse_account.address.to_bech32(), context.network_provider.proxy)
    if amount == 0:
        raise Exception(f"No amount found on {purse_account.address.to_bech32()}")
    print(f'Amount found on {purse_account.address.to_bech32()}: {amount}')

    # GET KNOWN CONTRACTS BEFORE UPGRADE -- THESE WILL BE REMOVED BY THE UPGRADE
    known_contracts = fees_collector_contract.get_known_contracts(proxy)

    # UPGRADE CONTRACT

    fees_collector_upgrade(bytecode_path)

    context.deployer_account.sync_nonce(proxy)
    fees_collector_contract.set_router_address(context.deployer_account, proxy, router_contract.address)
    chain_sim.advance_blocks(1)

    fees_collector_contract.add_admin(context.deployer_account, context.network_provider.proxy, [context.deployer_account.address])
    chain_sim.advance_blocks(1)

    # BEGIN SCENARIO

    def week_activity(week: int):
        # CLAIM REWARDS 1

        user_with_energy.sync_nonce(context.network_provider.proxy)
        hash = fees_collector_contract.claim_rewards(user_with_energy, proxy) #4 blocks
        chain_sim.advance_blocks(5)
        sleep(2)
        claim_ops = context.network_provider.get_tx_operations(hash, True)
        collector.add(f"CLAIM_USER_1_W_ENERGY_WEEK_{week}", claim_ops, "Claim from user1 with energy")

        user_without_energy.sync_nonce(context.network_provider.proxy)
        fees_collector_contract.claim_rewards(user_without_energy, proxy) #4 blocks
        chain_sim.advance_blocks(5)

        # SWAP

        purse_account.sync_nonce(proxy)
        swap = SwapFixedInputEvent(pair_contract.secondToken, amount//10, pair_contract.firstToken, 1)

        pair_contract.swap_fixed_input(context.network_provider, purse_account, swap)
        chain_sim.advance_blocks(5)

        # CONVERT TO MEX
        swappable_amount = get_available_amount_for_swap(pair_contract.secondToken)
        args = build_args_for_swap(pair_contract.secondToken, swappable_amount)
        context.deployer_account.sync_nonce(proxy)
        fees_collector_contract.swap_to_base_token(context.deployer_account, proxy, abi, args)
        chain_sim.advance_blocks(1)

        # CLAIM REWARDS 2

        user_with_energy_2.sync_nonce(context.network_provider.proxy)
        hash = fees_collector_contract.claim_rewards(user_with_energy_2, proxy) #4 blocks
        chain_sim.advance_blocks(5)
        sleep(2)
        claim_ops = context.network_provider.get_tx_operations(hash, True)
        collector.add(f"CLAIM_USER_2_W_ENERGY_WEEK_{week}", claim_ops, "Claim from user2 with energy")

    # FIRST WEEKS ------------------------------------------------------------

    weeks = 5
    for week in range(weeks):

        print(f"Week {week}")

        # WEEK ACTIVITY
        week_activity(week)

        # PASS THE WEEK
        chain_sim.advance_epochs_to_epoch(get_next_week_start_epoch(fees_collector_contract))

    # WEEK 5 SETUP ------------------------------------------------------------

    user_with_energy.sync_nonce(context.network_provider.proxy)
    hash = fees_collector_contract.claim_rewards(user_with_energy, proxy) #4 blocks
    chain_sim.advance_blocks(5)
    sleep(2)
    claim_ops = context.network_provider.get_tx_operations(hash, True)
    collector.add(f"RECLAIM_USER_1_W_ENERGY_WEEK_{week+1}", claim_ops, "Early claim from user1 with energy")

    # CLEANUP KNOWN TOKENS

    tokens = fees_collector_contract.get_reward_tokens(proxy)
    tokens.remove(energy_contract.base_token)
    tokens.remove(energy_contract.locked_token)

    fees_collector_contract.remove_reward_tokens(context.deployer_account, proxy, tokens)
    chain_sim.advance_blocks(1)

    # SET NEW BURN PERCENTAGE

    esdt_contract = ESDTContract(config.TOKENS_CONTRACT_ADDRESS)
    esdt_contract.set_special_role_token(context.deployer_account, context.network_provider.proxy,
                                                [energy_contract.base_token, fees_collector_contract.address, "ESDTRoleLocalBurn"])

    chain_sim.advance_blocks(5)

    fees_collector_contract.set_base_token_burn_percent(context.deployer_account, proxy, 5000)
    chain_sim.advance_blocks(1)

    # MODIFY FEES FOR KNOWN CONTRACTS

    for address in known_contracts:
        mod_contract = PairContract.load_contract_by_address(address)
        if mod_contract is None:
            continue
        mod_contract.add_fees_collector(context.deployer_account, proxy, [fees_collector_contract.address, 100000])
        chain_sim.advance_blocks(1)

    # WEEK ACTIVITY
    week_activity(week+1)

    chain_sim.advance_epochs_to_epoch(get_next_week_start_epoch(fees_collector_contract))

    # WEEK ACTIVITY
    week_activity(week+2)

BINARY COMPARE

In [ ]:
upgrade1_path = config.HOME / "wasm/fees-collector.wasm"
upgrade2_path = config.HOME / "wasm/fees-collector-rewards-per-epoch.wasm"

docker_path = config.HOME / "Projects/testing/full-stack-docker-compose/chain-simulator"
state_path = config.DEFAULT_WORKSPACE / "states"
args = Namespace(docker_path=str(docker_path), state_path=str(state_path))
    
collector = PhaseDictsCollector()

chain_sim, found_accounts = start_handler(args)
print(f'Loaded {len(found_accounts)} accounts')
sleep(10)

collector.set_phase("wasm-1")
test_scenario(upgrade1_path, collector)

chain_sim, found_accounts = start_handler(args)
print(f'Loaded {len(found_accounts)} accounts')
sleep(10)

collector.set_phase("wasm-2")
test_scenario(upgrade2_path, collector)

differences = collector.compare_all()
if differences:
    print("Found differences:")
    for diff in differences:
        print(f"- {diff}")
else:
    print("All comparisons passed!")

SINGLE BINARY

In [ ]:
upgrade_path = config.HOME / "Projects/dex/mx-exchange-sc/output-docker/fees-collector/fees-collector.wasm"

docker_path = config.HOME / "Projects/testing/full-stack-docker-compose/chain-simulator"
state_path = config.DEFAULT_WORKSPACE / "states"
args = Namespace(docker_path=str(docker_path), state_path=str(state_path))
    
collector = PhaseDictsCollector()

chain_sim, found_accounts = start_handler(args)
print(f'Loaded {len(found_accounts)} accounts')
sleep(10)

collector.set_phase("wasm-1")
test_scenario(upgrade_path, collector)

In [ ]:
collector.print_collections()

In [ ]:
swappable_amount = get_available_amount_for_swap(pair_contract.secondToken)
args = build_args_for_swap(pair_contract.secondToken, swappable_amount)
fees_collector_contract.swap_to_base_token(context.deployer_account, proxy, abi, args)
chain_sim.advance_blocks(1)

Fees and pairs setup

In [ ]:
from contracts.builtin_contracts import ESDTContract

esdt_contract = ESDTContract(config.TOKENS_CONTRACT_ADDRESS)
context.deployer_account.sync_nonce(context.network_provider.proxy)

tx_hash = esdt_contract.set_special_role_token(context.deployer_account, context.network_provider.proxy,
                                               [BASE_TOKEN, fees_collector_contract.address, "ESDTRoleLocalBurn"])

pair_contract.add_fees_collector(context.deployer_account, proxy, [fees_collector_contract.address, 100000])
mex_contract.add_fees_collector(context.deployer_account, proxy, [fees_collector_contract.address, 100000])
fees_collector_contract.set_base_token_burn_percent(context.deployer_account, proxy, 5000)
chain_sim.advance_blocks(1)

purse_account.sync_nonce(proxy)
swap = SwapFixedInputEvent(pair_contract.secondToken, amount//10, pair_contract.firstToken, 1)
pair_contract.swap_fixed_input(context.network_provider, purse_account, swap)
chain_sim.advance_blocks(5)

# CONVERT TO MEX

swappable_amount = get_available_amount_for_swap(pair_contract.secondToken)
args = build_args_for_swap(pair_contract.secondToken, swappable_amount)
fees_collector_contract.swap_to_base_token(context.deployer_account, proxy, abi, args)
chain_sim.advance_blocks(1)

# CLAIM REWARDS 2

user_with_energy_2.sync_nonce(context.network_provider.proxy)
fees_collector_contract.claim_rewards(user_with_energy_2, proxy) #4 blocks
chain_sim.advance_blocks(5)

In [ ]:
chain_sim.advance_blocks(1)       

In [ ]:
chain_sim.advance_epochs(7)

In [ ]:
chain_sim.advance_epochs_to_epoch(get_next_week_start_epoch(fees_collector_contract))

## Claim all

In [ ]:
from utils.utils_chain import WrapperAddress as Address, Account,string_to_hex
from utils.utils_tx import split_to_chunks
from multiversx_sdk.network_providers.resources import AccountStorage
from concurrent.futures import ThreadPoolExecutor
from multiversx_sdk import SmartContractTransactionsFactory, TransactionsFactoryConfig
from tools.runners.common_runner import get_default_signature
from utils.utils_generic import execute_parallel

def get_addresses_in_fees_collector(state: AccountStorage) -> list[str]:
    logger.debug(f'Getting addresses in fees collector')
    addresses_in_fees_collector = []
    for entry in state.entries:
        if "currentClaimProgress" in entry.key:
            for key in entry.raw.keys():
                hex_address = key.removeprefix(string_to_hex("currentClaimProgress"))
                addresses_in_fees_collector.append(Address.from_hex(hex_address).to_bech32())
    logger.debug(f'Found {len(addresses_in_fees_collector)} addresses in fees collector')
    return addresses_in_fees_collector

def fund_addresses_in_fees_collector(addresses: list[str]):
    logger.debug(f'Funding {len(addresses)} addresses in fees collector')
    count = 0
    user_addresses = []
    for address in addresses:
        if not Address(address).is_smart_contract():
            user_addresses.append(address)

    # with ThreadPoolExecutor(max_workers=100) as executor:
    #     args = [(address, 10 * 10**18) for address in user_addresses]
    #     executor.map(fund_chain_sim_user_w_egld, args)

    if config.CURRENT_ENV.value == "chainsim":
        fund_chain_sim_users_w_egld(user_addresses, 10 * 10**18)
    else:
        # fund only the users that have below 1 * 10**16 EGLD
        user_addresses = execute_parallel(lambda address: address if context.network_provider.proxy.get_account(WrapperAddress(address)).balance < 1 * 10**16 else None, user_addresses, 20)
        user_addresses = [address for address in user_addresses if address is not None]
        fund_shadowfork_users_w_egld(user_addresses, 1 * 10**17)

def get_nonces_for_addresses(addresses: list[str]) -> dict[str, int]:
    nonces = {}

    def get_nonce(address: str):
        nonce = context.network_provider.proxy.get_account(Address(address)).nonce
        nonces[address] = nonce
        return nonce
    
    execute_parallel(get_nonce, addresses, 20)

    return nonces

def claim_rewards_for_addresses(addresses: list[str], nonces: dict[str, int] = None) -> list[str]:
    logger.debug(f'Claiming rewards for {len(addresses)} addresses')
    hashes = []
    transactions = []
    signature = get_default_signature()

    chain_id = context.network_provider.proxy.get_network_config().chain_id
    config_tx = TransactionsFactoryConfig(chain_id=chain_id)

    for address in addresses:
        if Address(address).is_smart_contract():
            continue

        factory = SmartContractTransactionsFactory(config_tx)
        tx = factory.create_transaction_for_execute(
            Address(address),
            Address(fees_collector_contract.address),
            "claimRewards",
            150000000,
            [],
            0,
            [])
        tx.nonce = 0 if nonces is None else nonces[address]
        tx.signature = signature
        transactions.append(tx)

        if nonces is not None:
            nonces[address] += 1
    
    logger.debug(f"Starting to send {len(transactions)} transactions")
    transactions_chunks = split_to_chunks(transactions, 100)
    i = 0
    for chunk in transactions_chunks:
        try:
            num_sent, sent_hashes = context.network_provider.proxy.send_transactions(chunk)
            i += 1
            logger.debug(f"Sent {i} / {len(transactions) // 100 + 1 } chunks, {num_sent} / {len(chunk)} transactions")
            hashes.extend(sent_hashes)
            if config.CURRENT_ENV.value == "chainsim":
                chain_sim.advance_blocks(10)
        except Exception as e:
            logger.error(f"Error sending transactions: {e}")
            # retry sending the same chunk 3 times
            for _ in range(3):
                try:
                    num_sent, hashes = context.network_provider.proxy.send_transactions(chunk)
                    i += 1
                    logger.debug(f"Sent {i} / {len(transactions) // 100 + 1 } chunks, {num_sent} / {len(chunk)} transactions")
                    hashes.extend(hashes)
                    chain_sim.advance_blocks(20)
                    break
                except Exception as e:
                    logger.error(f"Error sending transactions: {e}")
                    continue
            else:
                logger.error(f"Failed to send transactions after 3 retries")
                raise e

    if config.CURRENT_ENV.value == "chainsim":
        chain_sim.advance_blocks(1000)
    return hashes

def check_if_hashes_are_failed(hashes: list[str]) -> list[str]:
    logger.debug(f'Checking if {len(hashes)} hashes are failed')
    failed_hashes = []

    def check_if_hash_is_failed(hash: bytes | str):
        try:
            status = context.network_provider.proxy.get_transaction_status(hash)
        except Exception as e:
            # retry 3 times
            for _ in range(3):
                try:
                    status = context.network_provider.proxy.get_transaction_status(hash)
                    break
                except Exception as e:
                    continue
            else:
                logger.error(f"Failed to get transaction status after 3 retries")
                return None
        
        if status.status == "pending":
            logger.warning(f"Hash {hash.hex() if isinstance(hash, bytes) else hash} is pending")
            return None
        if status.status != "success":
            return hash
        return None

    with ThreadPoolExecutor(max_workers=10) as executor:
        check_list = list(executor.map(check_if_hash_is_failed, reversed(hashes)))
    failed_hashes = [hash for hash in check_list if hash is not None]
    logger.debug(f'Found {len(failed_hashes)} failed hashes')
    return failed_hashes

def check_if_hashes_are_failed_via_api(hashes: list[str]) -> list[str]:
    logger.debug(f'Checking if {len(hashes)} hashes are failed')
    failed_hashes = []

    def check_if_hash_is_failed(hash: bytes | str):
        try:
            status = context.network_provider.api.get_transaction(hash).status
        except Exception as e:
            # retry 3 times
            for _ in range(3):
                try:
                    status = context.network_provider.api.get_transaction(hash).status
                    break
                except Exception as e:
                    continue
            else:
                logger.error(f"Failed to get transaction status after 3 retries")
                return None
        
        if status.is_failed:
            return hash
        return None

    processed = 0
    failed = 0
    total = len(hashes)
    
    def update_progress(future):
        nonlocal processed, failed
        processed += 1
        if future.result() is not None:
            failed += 1
        if processed % 100 == 0:
            print(f"Processed {processed}/{total} hashes, failed {failed}", end="\r")
    
    futures = []
    with ThreadPoolExecutor(max_workers=100) as executor:
        for hash in reversed(hashes):
            future = executor.submit(check_if_hash_is_failed, hash)
            future.add_done_callback(update_progress)
            futures.extend([future])
        
        check_list = [f.result() for f in futures]
    failed_hashes = [hash for hash in check_list if hash is not None]
    logger.debug(f'Found {len(failed_hashes)} failed hashes')
    return failed_hashes

In [ ]:
from utils.utils_scenarios import PhaseDictsCollector
from tools.chain_simulator_connector import ChainSimulator, start_handler
from utils.utils_chain import get_token_details_for_address, WrapperAddress as Address
from contracts.pair_contract import PairContract, SwapFixedInputEvent
from contracts.builtin_contracts import ESDTContract
from argparse import Namespace
from time import sleep

def prep_claim_all(bytecode_path: Path):
    pair_contract: PairContract= context.get_contracts(config.PAIRS_V2)[1]
    print(pair_contract.get_config_dict())

    # UPGRADE CONTRACT

    fees_collector_upgrade(bytecode_path)

    context.deployer_account.sync_nonce(proxy)
    fees_collector_contract.set_router_address(context.deployer_account, proxy, router_contract.address)
    chain_sim.advance_blocks(1)

    fees_collector_contract.add_admin(context.deployer_account, context.network_provider.proxy, [context.deployer_account.address])
    chain_sim.advance_blocks(1)

    # CONVERT TO MEX
    amount = get_available_amount_for_swap(pair_contract.secondToken)
    args = build_args_for_swap(pair_contract.secondToken, amount)
    context.deployer_account.sync_nonce(proxy)
    fees_collector_contract.swap_to_base_token(context.deployer_account, proxy, abi, args)
    chain_sim.advance_blocks(1)

    # REDISTRIBUTE MEX REWARDS
    fees_collector_contract.redistribute_rewards(context.deployer_account, context.network_provider.proxy)
    chain_sim.advance_blocks(1)

### Chainsim Executable

In [ ]:
# prep the test
prep_claim_all(wasm_path)

starting_epoch = context.network_provider.proxy.get_network_status().current_epoch
changing_epoch = get_next_week_start_epoch(fees_collector_contract)

state = context.network_provider.proxy.get_account_storage(Address(fees_collector_contract.address))
addresses_in_fees_collector = get_addresses_in_fees_collector(state)

fund_addresses_in_fees_collector(addresses_in_fees_collector)

# at this point, all user addresses (without SCS!!) are funded and have nonce 0
hashes = claim_rewards_for_addresses(addresses_in_fees_collector)

# check if any hash is failed
failed_hashes = check_if_hashes_are_failed_via_api(hashes)

print(f'Failed hashes: {len(failed_hashes)}')
print(failed_hashes)

ending_epoch = context.network_provider.proxy.get_network_status().current_epoch
print(f"Starting epoch: {starting_epoch}, Week changing in epoch: {changing_epoch}, Ending epoch: {ending_epoch}")

### Shadowfork Executable

preps

In [ ]:
starting_epoch = context.network_provider.proxy.get_network_status().current_epoch
changing_epoch = get_next_week_start_epoch(fees_collector_contract)

state = context.network_provider.proxy.get_account_storage(Address(fees_collector_contract.address))
addresses_in_fees_collector = get_addresses_in_fees_collector(state)

In [ ]:
fund_addresses_in_fees_collector(addresses_in_fees_collector)

In [ ]:
nonces = get_nonces_for_addresses(addresses_in_fees_collector)

In [ ]:
advance_to_next_week(fees_collector_contract)

claims

In [ ]:
# at this point, all user addresses (without SCS!!) are funded and have nonce 0
hashes = claim_rewards_for_addresses(addresses_in_fees_collector, nonces)

In [ ]:
from pprint import pprint

# wait for network to be idle
block = context.network_provider.proxy.get_latest_block(1)
block_gas = block.raw.get("scheduledData", {}).get("gasProvided", 0)
while block_gas > 1000000:
    sleep(12)
    block = context.network_provider.proxy.get_latest_block(1)
    block_gas = block.raw.get("scheduledData").get("gasProvided")

print(f"Network processing is over!")

checks

In [ ]:
# check if any hash is failed
failed_hashes = check_if_hashes_are_failed_via_api(hashes)

print(f'Failed hashes: {len(failed_hashes)}')
print(failed_hashes)

ending_epoch = context.network_provider.proxy.get_network_status().current_epoch
print(f"Started in epoch: {starting_epoch}, Week changing in epoch: {changing_epoch}, Ending epoch: {ending_epoch}")

In [ ]:
for token in context.deploy_structure.tokens:
    print(f"{token}: {get_available_amount_for_swap_via_view(token)}")

swaps and redistribution

In [ ]:
swaps_for_deposits(50)

In [ ]:
swap_all_tokens_to_base_token()
if config.CURRENT_ENV.value == "chainsim":
    chain_sim.advance_blocks(6)

In [ ]:
fees_collector_contract.redistribute_rewards(context.deployer_account, context.network_provider.proxy)
if config.CURRENT_ENV.value == "chainsim":
    chain_sim.advance_blocks(1)

manual

In [ ]:
epoch = get_next_week_start_epoch(fees_collector_contract)
chain_sim.advance_epochs_to_epoch(epoch)

In [ ]:
print(context.network_provider.proxy.get_network_status().current_epoch)

In [ ]:
# prep the test
prep_claim_all(wasm_path)

In [ ]:
from contracts.simple_lock_energy_contract import SimpleLockEnergyContract
energy_contract: SimpleLockEnergyContract = context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0]

energy_contract.get_energy_for_user(context.network_provider.proxy, "erd1ll0lqrklv4f09jdwmd3l9gktwhzxzrw3uvrnqzxg2tenlnpyq8sqm8h7yv")

In [ ]:
from utils.utils_chain import Account, WrapperAddress

user = "erd1ll0lqrklv4f09jdwmd3l9gktwhzxzrw3uvrnqzxg2tenlnpyq8sqm8h7yv"

fund_chain_sim_users_w_egld([user], 10 * 10**18)

user_account = Account(pem_file=config.DEFAULT_ACCOUNTS)
user_account.address = WrapperAddress(user)
user_account.sync_nonce(context.network_provider.proxy)

fees_collector_contract.claim_rewards(user_account, context.network_provider.proxy)
chain_sim.advance_blocks(6)

In [ ]:
fees_collector_contract.redistribute_rewards(context.deployer_account, context.network_provider.proxy)
chain_sim.advance_blocks(1)

## Redistribute MEX rewards

In [ ]:
from time import sleep

_, balance, _ = get_token_details_for_address("MEX-455c57", fees_collector_contract.address, context.network_provider.proxy)
undistributed = get_available_amount_for_swap("MEX-455c57")
current_accumulated = fees_collector_contract.get_accumulated_fees(context.network_provider.proxy, "MEX-455c57")

fees_collector_contract.redistribute_rewards(context.deployer_account, context.network_provider.proxy)
chain_sim.advance_blocks(1)
sleep(6)

resulting_accumulated = fees_collector_contract.get_accumulated_fees(context.network_provider.proxy, "MEX-455c57")

assert resulting_accumulated == current_accumulated + undistributed

Claims counting consistency

In [ ]:
print(f"resulting accumulated: {resulting_accumulated}")
print(f"expected accumulated: {current_accumulated + undistributed}")
print(f"first accumulated: {current_accumulated}")
print(f"first undistributed: {undistributed}")
print(f"first balance: {balance}")
_, amount, _ = get_token_details_for_address("MEX-455c57", fees_collector_contract.address, context.network_provider.proxy)
print(f"current balance: {amount}")
print(get_available_amount_for_swap("MEX-455c57"))

In [ ]:
from pprint import pprint
event = collector.collections["wasm-1"]

current_week = fees_collector_contract.get_current_week(context.network_provider.proxy)
print(f"Current week: {current_week}")

collected_weeks = 6
for week in range(collected_weeks, collected_weeks - 4, -1):
    searched_week_string = f"WEEK_{week}"
    claims = [collection for collection in collector.dict_types if "CLAIM" in collection and collection.endswith(searched_week_string)]

    print(f"Found {len(claims)} claims for collected week {week}")

    sum_of_claims = 0
    for claim in claims:
        operations, _ = event[claim][0]
        # pprint(operations)
        for operation in operations:
            if operation.get('action') == 'transfer' and operation.get('identifier') == "MEX-455c57" and operation.get('sender') == fees_collector_contract.address:
                sum_of_claims += int(operation.get('value'))
                print(f"Claimed {operation.get('value')} MEX")

    print(f"Sum of claims for week {week}: {sum_of_claims}")
    week_to_check = current_week - collected_weeks + week - 1
    print(f"Contract claims for week {week_to_check}: {fees_collector_contract.get_rewards_claimed(context.network_provider.proxy, week_to_check, "MEX-455c57")}")
    print()

## Consistent swap amounts

In [ ]:
users = users_init(found_accounts)

purse_account = next(user for user in users if user.address.to_bech32() == "erd146exyad7pn95pru78egj07nnyfgnyeaytxte33nxxd24g55uccgs77rr7d")
user_without_energy = next(user for user in users if user.address.to_bech32() == "erd146exyad7pn95pru78egj07nnyfgnyeaytxte33nxxd24g55uccgs77rr7d")
user_with_energy = next(user for user in users if user.address.to_bech32() == "erd1emxytu3umnzm4k2cn2xmtppy8j3dm3lnsjhfzkul8gd5a4xxuk3qsl4xjw")
user_with_energy_2 = next(user for user in users if user.address.to_bech32() == "erd1adljw932qra4sf5mpxjyzelmf4lykwt5ppxlre59utjcpc22uhms2qxcqx")

pair_contract: PairContract= context.get_contracts(config.PAIRS_V2)[1]
print(pair_contract.get_config_dict())

_, amount, _ = get_token_details_for_address(pair_contract.secondToken, purse_account.address.to_bech32(), context.network_provider.proxy)
if amount == 0:
    raise Exception(f"No amount found on {purse_account.address.to_bech32()}")
print(f'Amount found on {purse_account.address.to_bech32()}: {amount}')

# GET KNOWN CONTRACTS BEFORE UPGRADE -- THESE WILL BE REMOVED BY THE UPGRADE
known_contracts = fees_collector_contract.get_known_contracts(proxy)

# UPGRADE CONTRACT

fees_collector_upgrade(wasm_path, contract_code_hash)

context.deployer_account.sync_nonce(proxy)
fees_collector_contract.set_router_address(context.deployer_account, proxy, router_contract.address)
chain_sim.advance_blocks(1)

fees_collector_contract.add_admin(context.deployer_account, context.network_provider.proxy, [context.deployer_account.address])
chain_sim.advance_blocks(1)

# BEGIN SCENARIO

In [ ]:
def week_activity(week: int):
    from contracts.pair_contract import SwapFixedInputEvent
    # CLAIM REWARDS 1

    user_with_energy.sync_nonce(context.network_provider.proxy)
    hash = fees_collector_contract.claim_rewards(user_with_energy, proxy) #4 blocks
    chain_sim.advance_blocks(5)

    user_without_energy.sync_nonce(context.network_provider.proxy)
    fees_collector_contract.claim_rewards(user_without_energy, proxy) #4 blocks
    chain_sim.advance_blocks(5)

    # SWAP

    purse_account.sync_nonce(proxy)
    swap = SwapFixedInputEvent(pair_contract.secondToken, amount//10, pair_contract.firstToken, 1)

    pair_contract.swap_fixed_input(context.network_provider, purse_account, swap)
    chain_sim.advance_blocks(5)

    # CONVERT TO MEX
    swappable_amount = get_available_amount_for_swap(pair_contract.secondToken)
    args = build_args_for_swap(pair_contract.secondToken, swappable_amount)
    context.deployer_account.sync_nonce(proxy)
    fees_collector_contract.swap_to_base_token(context.deployer_account, proxy, abi, args)
    chain_sim.advance_blocks(1)

    # CLAIM REWARDS 2

    user_with_energy_2.sync_nonce(context.network_provider.proxy)
    hash = fees_collector_contract.claim_rewards(user_with_energy_2, proxy) #4 blocks
    chain_sim.advance_blocks(5)

In [ ]:
print(f"theoretical available amount: {get_available_amount_for_swap(pair_contract.secondToken)}")
print(f'contract available amount: {get_available_amount_for_swap_via_view(pair_contract.secondToken)}')

In [ ]:
# 1015904170
calc = 723716468 - 723665676
print(calc)

In [ ]:
user_with_energy.sync_nonce(context.network_provider.proxy)
hash = fees_collector_contract.claim_rewards(user_with_energy, proxy) #4 blocks
chain_sim.advance_blocks(5)

In [ ]:
from contracts.pair_contract import SwapFixedInputEvent
purse_account.sync_nonce(proxy)
swap = SwapFixedInputEvent(pair_contract.secondToken, amount//10, pair_contract.firstToken, 1)
pair_contract.swap_fixed_input(context.network_provider, purse_account, swap)
chain_sim.advance_blocks(5)

In [ ]:
# WEEK ACTIVITY
week_activity(1)

In [ ]:
# PASS THE WEEK
chain_sim.advance_epochs_to_epoch(get_next_week_start_epoch(fees_collector_contract))

## Upgrade timing

In [ ]:
def select_users_with_energy_from_list(addresses: list[str], selected_number: int) -> list[str]:
    current_week = fees_collector_contract.get_current_week(context.network_provider.proxy)
    selected_addresses = []
    for address in addresses:
        # get energy for the previous week (since the current week is not yet over, some users might've not claimed yet)
        energy = fees_collector_contract.get_user_energy_for_week(address, context.network_provider.proxy, current_week - 1)
        if energy.get("amount", 0) <= 0:
            continue
        if WrapperAddress(address).is_smart_contract():
            continue
        if context.network_provider.proxy.get_account(WrapperAddress(address)).balance < 1 * 10 ** 16:
            continue
        selected_addresses.append(address)
        if len(selected_addresses) == selected_number:
            break
    if len(selected_addresses) < selected_number:
        raise Exception(f"Not enough addresses with energy. Found {len(selected_addresses)} out of {selected_number}")
    return selected_addresses

def test_upgrade_timing():
    print(f"Starting test in week: {fees_collector_contract.get_current_week(context.network_provider.proxy)}")
    state = context.network_provider.proxy.get_account_storage(Address(fees_collector_contract.address))
    addresses_in_fees_collector = get_addresses_in_fees_collector(state)
    fund_addresses_in_fees_collector(addresses_in_fees_collector)

    # Select an user with energy to claim rewards
    user = select_users_with_energy_from_list(addresses_in_fees_collector, 1)[0]
    # claim rewards
    user_account = Account(pem_file=config.DEFAULT_ACCOUNTS)
    user_account.address = WrapperAddress(user)
    user_account.sync_nonce(context.network_provider.proxy)
    fees_collector_contract.claim_rewards(user_account, context.network_provider.proxy)
    chain_sim.advance_blocks(5)

    # UPGRADE CONTRACT
    fees_collector_upgrade(wasm_path)

    context.deployer_account.sync_nonce(proxy)
    fees_collector_contract.set_router_address(context.deployer_account, proxy, router_contract.address)
    chain_sim.advance_blocks(1)

    fees_collector_contract.add_admin(context.deployer_account, context.network_provider.proxy, [context.deployer_account.address])
    chain_sim.advance_blocks(1)

    swaps_for_deposits(50)

    # CONVERT TO MEX
    swap_all_tokens_to_base_token()
    if config.CURRENT_ENV.value == "chainsim":
        chain_sim.advance_blocks(6)

    # REDISTRIBUTE MEX REWARDS
    fees_collector_contract.redistribute_rewards(context.deployer_account, context.network_provider.proxy)
    chain_sim.advance_blocks(1)

    # PASS THE WEEK
    from contracts.builtin_contracts import SFControlContract
    changing_epoch = get_next_week_start_epoch(fees_collector_contract)
    chain_sim.advance_epochs_to_epoch(changing_epoch)
    while context.network_provider.proxy.get_network_status().current_epoch < changing_epoch:
        sleep(6)
    print(f"Fast forwarded to {changing_epoch}")

    # Claim rewards for the user with energy
    fees_collector_contract.claim_rewards(user_account, context.network_provider.proxy)
    chain_sim.advance_blocks(5)

In [ ]:
test_upgrade_timing()

## Claim some

Select some accounts

In [ ]:
state = context.network_provider.proxy.get_account_storage(Address(fees_collector_contract.address))
addresses_in_fees_collector = get_addresses_in_fees_collector(state)
users = select_users_with_energy_from_list(addresses_in_fees_collector, 6)

In [ ]:
print(users)

Claims up to index

In [ ]:
users_to_claim = 6
for index, user in enumerate(users):
    if index >= users_to_claim:
        break
    user_account = Account(pem_file=config.DEFAULT_ACCOUNTS)
    user_account.address = WrapperAddress(user)
    user_account.sync_nonce(context.network_provider.proxy)
    fees_collector_contract.claim_rewards(user_account, context.network_provider.proxy)
    if config.CURRENT_ENV.value == "chainsim":
        chain_sim.advance_blocks(5)

In [ ]:
swaps_for_deposits(50)
swap_all_tokens_to_base_token()
fees_collector_contract.redistribute_rewards(context.deployer_account, context.network_provider.proxy)

In [ ]:
advance_to_next_week(fees_collector_contract)

# Isolated claims

## Init

In [ ]:
from utils.contract_data_fetchers import FeeCollectorContractDataFetcher
from utils.utils_chain import hex_to_string

def deploy_isolated_fees_collector() -> FeesCollectorContract:
    context.deployer_account.sync_nonce(context.network_provider.proxy)
    isolated_fees_collector = FeesCollectorContract("")
    old_wasm = "https://github.com/multiversx/mx-exchange-sc/releases/download/v3.0.8/fees-collector.wasm"
    _, address = isolated_fees_collector.contract_deploy(context.deployer_account, context.network_provider.proxy, old_wasm, 
        [energy_contract.locked_token, energy_contract.address])
    if config.CURRENT_ENV.value == "chainsim":
        chain_sim.advance_blocks(1)
    isolated_fees_collector.address = address

    isolated_fees_collector.set_energy_factory_address(context.deployer_account, context.network_provider.proxy, energy_contract.address)
    isolated_fees_collector.set_locking_address(context.deployer_account, context.network_provider.proxy, energy_contract.address)
    isolated_fees_collector.set_lock_epochs(context.deployer_account, context.network_provider.proxy, 1440)
    isolated_fees_collector.set_locked_tokens_per_block(context.deployer_account, context.network_provider.proxy, 3 * 10**18)
    energy_contract.add_sc_to_whitelist(context.deployer_account, context.network_provider.proxy, isolated_fees_collector.address)
    if config.CURRENT_ENV.value == "chainsim":
        chain_sim.advance_blocks(3)
    return isolated_fees_collector

def move_fees_collector_to_isolated_address(fees_collector: FeesCollectorContract, isolated_fees_collector: FeesCollectorContract):
    context.deployer_account.sync_nonce(context.network_provider.proxy)
    know_contracts = fees_collector.get_known_contracts(context.network_provider.proxy)
    for address in know_contracts:
        mod_contract = PairContract.load_contract_by_address(address)
        if mod_contract is None:
            continue
        mod_contract.add_fees_collector(context.deployer_account, context.network_provider.proxy, [isolated_fees_collector.address, 50000])
        isolated_fees_collector.add_known_contracts(context.deployer_account, context.network_provider.proxy, [address])
        if config.CURRENT_ENV.value == "chainsim":
            chain_sim.advance_blocks(1)
    
    data_fetcher = FeeCollectorContractDataFetcher(Address(fees_collector.address), context.network_provider.proxy.url)
    hex_results = data_fetcher.get_data("getAllTokens")
    if not hex_results:
        raise Exception("No tokens found")
    tokens = [hex_to_string(token) for token in hex_results]
    isolated_fees_collector.add_known_tokens(context.deployer_account, context.network_provider.proxy, tokens)
    if config.CURRENT_ENV.value == "chainsim":
        chain_sim.advance_blocks(1)

def prep_claim_addresses():
    from utils.utils_tx import ESDTToken
    user_addresses = [
        WrapperAddress.from_hex("1ba007a4d21e3252d297633f100c3f2e8dbe70d8b977a77ee6368507cd4c8d01").bech32(),
        WrapperAddress.from_hex("1ba007a4d21e3252d297633f100c3f2e8dbe70d8b977a77ee6368507cd4c8d11").bech32(),
        WrapperAddress.from_hex("1ba007a4d21e3252d297633f100c3f2e8dbe70d8b977a77ee6368507cd4c8d21").bech32(),
        ]
    chain_sim.fund_users_w_egld(user_addresses, 10 * 10**18)
    chain_sim.fund_users_w_esdt_from_mainnet(user_addresses, energy_contract.base_token, 10 * 10**18)

    user_accounts = []
    for user in user_addresses:
        user_account = Account(pem_file=config.DEFAULT_ACCOUNTS)
        user_account.address = WrapperAddress(user)
        user_account.sync_nonce(context.network_provider.proxy)
        
        tokens = [ESDTToken(energy_contract.base_token, 0, 10 * 10**18)]
        energy_contract.lock_tokens(user_account, context.network_provider.proxy, [tokens, 1440])
        fees_collector_contract.claim_rewards(user_account, context.network_provider.proxy)
        if config.CURRENT_ENV.value == "chainsim":
            chain_sim.advance_blocks(2)
        user_accounts.append(user_account)
    return user_accounts

## Execute

deploy and setup

In [ ]:
# executable on chainsim
isolated_fees_collector = deploy_isolated_fees_collector()

move_fees_collector_to_isolated_address(fees_collector_contract, isolated_fees_collector)
fees_collector_contract = isolated_fees_collector

prep claim addresses

In [ ]:
swaps_for_deposits(20)
users = prep_claim_addresses()

4 weeks priming

In [ ]:
weeks_to_claim = 4
for week in range(weeks_to_claim):
    advance_to_next_week(fees_collector_contract)
    for user in users[:2]:
        fees_collector_contract.claim_rewards(user, context.network_provider.proxy)
        chain_sim.advance_blocks(2)
    swaps_for_deposits(20)

upgrade week

In [ ]:
# UPGRADE WEEK
advance_to_next_week(fees_collector_contract)
# claim one user before upgrade
fees_collector_contract.claim_rewards(users[0], context.network_provider.proxy)
chain_sim.advance_blocks(1)

swaps_for_deposits(10)

# upgrade and setup
fees_collector_upgrade(wasm_path)
fees_collector_contract.set_router_address(context.deployer_account, proxy, router_contract.address)
chain_sim.advance_blocks(1)
fees_collector_contract.add_admin(context.deployer_account, context.network_provider.proxy, [context.deployer_account.address])
chain_sim.advance_blocks(1)

# deposit some more tokens & swap them
swaps_for_deposits(10)
swap_all_tokens_to_base_token()
fees_collector_contract.redistribute_rewards(context.deployer_account, context.network_provider.proxy)
chain_sim.advance_blocks(1)

# claim one more user
fees_collector_contract.claim_rewards(users[1], context.network_provider.proxy)
chain_sim.advance_blocks(1)

first week after upgrade

In [ ]:
# FIRST WEEK AFTER UPGRADE
advance_to_next_week(fees_collector_contract)

fees_collector_contract.claim_rewards(users[0], context.network_provider.proxy)
chain_sim.advance_blocks(1)

# deposit some more tokens & swap them
swaps_for_deposits(20)
swap_all_tokens_to_base_token()
fees_collector_contract.redistribute_rewards(context.deployer_account, context.network_provider.proxy)
chain_sim.advance_blocks(1)

# claim one more user
fees_collector_contract.claim_rewards(users[1], context.network_provider.proxy)
chain_sim.advance_blocks(1)

second week after upgrade

In [ ]:
# SECOND WEEK AFTER UPGRADE
advance_to_next_week(fees_collector_contract)

for user in users:
    fees_collector_contract.claim_rewards(user, context.network_provider.proxy)
    chain_sim.advance_blocks(1)

In [ ]:
swaps_for_deposits(20)

In [ ]:
swap_all_tokens_to_base_token()
fees_collector_contract.redistribute_rewards(context.deployer_account, context.network_provider.proxy)
chain_sim.advance_blocks(5)

In [ ]:
chain_sim.advance_blocks(5)

In [ ]:
swap_all_tokens_to_base_token()
fees_collector_contract.redistribute_rewards(context.deployer_account, context.network_provider.proxy)
chain_sim.advance_blocks(1)

In [ ]:
get_available_amount_for_swap_via_view("ASH-a642d1")

# Manual upgrade

In [ ]:
import sys
from pathlib import Path
from multiversx_sdk import ApiNetworkProvider, ProxyNetworkProvider
from contracts.fees_collector_contract import FeesCollectorContract
import os
import importlib

os.environ["MX_DEX_ENV"] = "devnet"

sys.path.append(str(Path.cwd().parent.parent.absolute()))
import config
importlib.reload(config)
from context import Context
from utils.utils_chain import WrapperAddress

context = Context()

wasm_path = config.HOME / "Projects/dex/mx-exchange-sc/output-docker/fees-collector/fees-collector.wasm"
contract_code_hash = "dde4f3d826d5bd801f318dd1e74637353146d6ee2111f3ad534ef28a868a3fcd"
admin = "erd1rwsq0fxjrce9955hvvl3qrpl96xmuuxch9m6wlhxx6zs0n2v3hvqyu4lm5"
fees_collector_contract: FeesCollectorContract = context.get_contracts(config.FEES_COLLECTORS)[0]

In [ ]:
fees_collector_upgrade(wasm_path, contract_code_hash)
if config.CURRENT_ENV.value == "chainsim":
    chain_sim.advance_blocks(1)

In [ ]:
# devnet upgrade
if config.CURRENT_ENV.value == "devnet":
    from multiversx_sdk import CodeMetadata
    from utils.utils_tx import upgrade_call
    from utils.utils_chain import WrapperAddress
    from pprint import pprint

    metadata = CodeMetadata(upgradeable=True, payable_by_contract=True, readable=True)
    gas_limit = 200000000

    blocks = int(context.network_provider.proxy.get_network_config().raw['erd_rounds_per_epoch'])
    pprint(blocks)
    arguments = [blocks]

    upgrade_call("devnet_fees_collector", context.network_provider.proxy, gas_limit, context.deployer_account, WrapperAddress(fees_collector_contract.address), wasm_path, metadata, arguments)

In [ ]:
# context.deployer_account.sync_nonce(context.network_provider.proxy)
fees_collector_contract.set_router_address(context.deployer_account, context.network_provider.proxy, context.get_contracts(config.ROUTER_V2)[0].address)
if config.CURRENT_ENV.value == "chainsim":
    chain_sim.advance_blocks(1)

In [ ]:
# context.deployer_account.sync_nonce(context.network_provider.proxy)
admin = "erd1p536zp9ndvlacsxkk6tv50uf9ql85vfhdkl9z7vv9xuw6q7rfrfsxqq72p"
fees_collector_contract.add_admin(context.deployer_account, context.network_provider.proxy, [admin])
if config.CURRENT_ENV.value == "chainsim":
    chain_sim.advance_blocks(1)

# Mainnet release procedure

In [ ]:
import sys
from pathlib import Path
from multiversx_sdk import ApiNetworkProvider, ProxyNetworkProvider
from contracts.fees_collector_contract import FeesCollectorContract
import os
import importlib

os.environ["MX_DEX_ENV"] = "shadowfork4"

sys.path.append(str(Path.cwd().parent.parent.absolute()))
import config
importlib.reload(config)
from context import Context
from utils.utils_chain import WrapperAddress

context = Context()

wasm_path = "https://github.com/multiversx/mx-exchange-sc/releases/download/v3.4.1-rc4/fees-collector.wasm"
contract_code_hash = "4a7f6baf4aeebd1c9892b6bafd74ee3548b04be17d23fc0d554318015568d0a9"

swap_bot = "erd1rwsq0fxjrce9955hvvl3qrpl96xmuuxch9m6wlhxx6zs0n2v3hvqyu4lm5"
fees_collector_contract: FeesCollectorContract = context.get_contracts(config.FEES_COLLECTORS)[0]

## Upgrade time

Upgrade and setup

In [ ]:
context.deployer_account.sync_nonce(context.network_provider.proxy)
tx_hash = fees_collector_contract.contract_upgrade(context.deployer_account, context.network_provider.proxy, wasm_path, 
                                        [], 
                                        no_init=True)
sleep(6)
code_hash = context.network_provider.proxy.get_account(WrapperAddress(fees_collector_contract.address)).contract_code_hash.hex()
assert code_hash == contract_code_hash

In [ ]:
fees_collector_contract.set_router_address(context.deployer_account, context.network_provider.proxy, context.get_contracts(config.ROUTER_V2)[0].address)

Whitelist the swapping bot

In [ ]:
fees_collector_contract.add_admin(context.deployer_account, context.network_provider.proxy, [swap_bot])

## 5th week after upgrade

Cleanup known tokens list

In [ ]:
tokens = fees_collector_contract.get_reward_tokens(proxy)
tokens.remove(energy_contract.base_token)
tokens.remove(energy_contract.locked_token)
context.deployer_account.sync_nonce(context.network_provider.proxy)
fees_collector_contract.remove_reward_tokens(context.deployer_account, context.network_provider.proxy, tokens)

Set burn role for MEX

In [ ]:
from contracts.builtin_contracts import ESDTContract
esdt_contract = ESDTContract(config.TOKENS_CONTRACT_ADDRESS)
esdt_contract.set_special_role_token(context.deployer_account, context.network_provider.proxy,
                                            [energy_contract.base_token, fees_collector_contract.address, "ESDTRoleLocalBurn"])

Modify fees for known contracts that were depositing fees

In [ ]:
known_contracts = fees_collector_contract.get_known_contracts(proxy)
print(f"Retrieved {len(known_contracts)} known contracts.")

known_pairs = []
for address in known_contracts:
    mod_contract = PairContract.load_contract_by_address(address)
    if mod_contract is None:
        continue
    mod_contract.add_fees_collector(context.deployer_account, context.network_provider.proxy, [fees_collector_contract.address, 100000])
    known_pairs.append(mod_contract.address)

Enable burn on fees collector

In [ ]:
fees_collector_contract.set_base_token_burn_percent(context.deployer_account, context.network_provider.proxy, 5000)

Cleanup the known contracts list

In [ ]:
fees_collector_contract.remove_known_contracts(context.deployer_account, context.network_provider.proxy, known_pairs)